In [ ]:
# CÓDIGO 1.1)

import pandas as pd # Biblioteca para manipulação de dados.
import matplotlib.pyplot as plt # Bibliotecas para visualização de dados.
import seaborn as sns # Bibliotecas para visualização de dados.
from xgboost import XGBRegressor # Classe do XGBoost para tarefas de regressão.
from sklearn.model_selection import train_test_split, cross_val_score # Função para realizar validação cruzada.
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error# Função para calcular o erro quadrático médio.
import numpy as np # Biblioteca para operações matemáticas adicionais.


# Carrega o dataset do caminho especificado usando pandas.
dataset_path = '/content/Jubarte_SEx_Ex_Ez_EW01_2002_Poly3_AI_MCD_5_5_6_0_14_0.csv' # AMPMOD , PHASEMOD
#dataset_path = '/content/rotated_dataset5.csv' # REAL E IMAGINARIO
data = pd.read_csv(dataset_path)


data = data[['AmpMod', 'PhaseMod', 'Freq']] # COLUNAS SELECIONADAS (AmpMod, PhaseMod, Frequencia)
#data = data[['AmpMod', 'PhaseMod', 'Offset', 'Freq']] # COLUNAS SELECIONADAS ( AmpMod, PhaseMod, Offset, Frequencia)
#data = data[['Real', 'Imaginario', 'Freq']] # COLUNAS SELECIONADAS ( Real, Imaginario, Frequencia)


X = data[['AmpMod', 'PhaseMod']]  # Features -  (AmpMod e PhaseMod).
#X = data[['AmpMod', 'PhaseMod', 'Offset']]  # Features -  (AmpMod , PhaseMod , Offset).
#X = data[['Real', 'Imaginario']]  # Features -  (Real , Imaginario).
y = data['Freq']                  # Target - (Freq).

# Divisão em conjuntos de treino (80%) e teste (20%), # shuffle=True embaralha os dados antes de dividir.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

model = XGBRegressor(random_state=42) # Inicializa o modelo de regressão do XGBoost.

# Treina o modelo XGBRegressor com os dados de treino.
model.fit(X_train, y_train) # Treina o modelo usando os dados de treino (X_train e y_train).

In [ ]:
y_pred = model.predict(X_test)

fig, ax = plt.subplots(1,2 , figsize=(15, 8))

ax[0].plot(y_test.values[:100], label='Real')
ax[0].plot(y_pred[:100], label='Predito')
ax[0].set_xlabel('Amostras')
ax[0].set_ylabel('Frequência')

ax[1].hist(y_test.values, label='Real')
ax[1].hist(y_pred, label='Predito', fill=False)
ax[1].set_xlabel('Amostras')
ax[1].set_ylabel('Frequência')
plt.legend()

In [ ]:
# Calcula o erro quadrático médio (MSE) das previsões.
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Calcula o erro quadrático médio (RMSE) das previsões.
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error: {rmse}")

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R-squared (R²): {r2}")
print(f"Root Mean Squared Error (RMSE): {rmse}")

In [ ]:
# Visualizando a importância das features.
feature_importances = model.feature_importances_
features = ['AmpMod', 'PhaseMod']

# Exibindo a importância das features no console
print("Importância das features:")
for name, val in zip(features, feature_importances):
    print(f'{name}: {val:.6f}')


In [ ]:
# Visualizando a importância das features.
feature_importances = model.feature_importances_
#features = ['AmpMod', 'PhaseMod', 'Offset']
features = ['AmpMod', 'PhaseMod']
#features = ['Real', 'Imaginario']

# Cria um DataFrame para a visualização.
importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importances})

# Ordena o DataFrame pela importância das features.
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Plota a importância das features.
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=importance_df, palette='rainbow')
plt.title('Importância das Features')
plt.xlabel('Importância')
plt.ylabel('Feature')
plt.show()

In [ ]:
from sklearn.model_selection import cross_val_score

# Realiza a validação cruzada k-fold com 5 folds.
kfold = 5
cv_results = cross_val_score(model, X, y, cv=kfold, scoring='neg_mean_squared_error')

# Converte os resultados para valores positivos e calcula a média.
cv_mse = np.mean(np.abs(cv_results))
cv_rmse = np.sqrt(cv_mse)

print(f"Cross-Validation MSE (mean): {cv_mse}")
print(f"Cross-Validation RMSE (mean): {cv_rmse}")

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

# Inicializar os modelos
rf_model = RandomForestRegressor(random_state=42)
lr_model = LinearRegression()

# Treinar e prever com RandomForest
rf_model.fit(X_train, y_train)
rf_y_pred = rf_model.predict(X_test)

# Treinar e prever com LinearRegression
lr_model.fit(X_train, y_train)
lr_y_pred = lr_model.predict(X_test)

# Avaliar o desempenho de ambos os modelos
for model_name, pred in zip(['XGBoost', 'RandomForest', 'LinearRegression'], [y_pred, rf_y_pred, lr_y_pred]):
    mse = mean_squared_error(y_test, pred)
    mae = mean_absolute_error(y_test, pred)
    r2 = r2_score(y_test, pred)
    rmse = np.sqrt(mse)
    print(f"{model_name} - MSE: {mse}, MAE: {mae}, R²: {r2}, RMSE: {rmse}")

In [ ]:
from scipy.stats import wilcoxon

# Prever novamente com os diferentes modelos para obter todos os erros
y_pred_xgb = model.predict(X_test)  # Previsões do modelo XGBoost
y_pred_rf = rf_model.predict(X_test)  # Previsões do modelo RandomForest
y_pred_lr = lr_model.predict(X_test)  # Previsões do modelo LinearRegression

# Calcular os erros absolutos (diferenças entre valores reais e previstos) para cada modelo
errors_xgb = np.abs(y_test - y_pred_xgb)
errors_rf = np.abs(y_test - y_pred_rf)
errors_lr = np.abs(y_test - y_pred_lr)

# Realizar o teste de Wilcoxon entre XGBoost e RandomForest
stat, p_value_xgb_rf = wilcoxon(errors_xgb, errors_rf)
print(f"Wilcoxon test between XGBoost and RandomForest: p-value = {p_value_xgb_rf}")

# Realizar o teste de Wilcoxon entre XGBoost e LinearRegression
stat, p_value_xgb_lr = wilcoxon(errors_xgb, errors_lr)
print(f"Wilcoxon test between XGBoost and LinearRegression: p-value = {p_value_xgb_lr}")

# Realizar o teste de Wilcoxon entre RandomForest e LinearRegression
stat, p_value_rf_lr = wilcoxon(errors_rf, errors_lr)
print(f"Wilcoxon test between RandomForest and LinearRegression: p-value = {p_value_rf_lr}")

# Interpretação dos resultados
alpha = 0.05
print("\nInterpretação dos resultados:")
if p_value_xgb_rf < alpha:
    print("Diferença entre XGBoost e RandomForest é estatisticamente significativa.")
else:
    print("Diferença entre XGBoost e RandomForest não é estatisticamente significativa.")

if p_value_xgb_lr < alpha:
    print("Diferença entre XGBoost e LinearRegression é estatisticamente significativa.")
else:
    print("Diferença entre XGBoost e LinearRegression não é estatisticamente significativa.")

if p_value_rf_lr < alpha:
    print("Diferença entre RandomForest e LinearRegression é estatisticamente significativa.")
else:
    print("Diferença entre RandomForest e LinearRegression não é estatisticamente significativa.")